Importing OpenAI API Key

In [ ]:
import os
from dotenv import load_dotenv,find_dotenv
from langchain_openai import OpenAI

load_dotenv(find_dotenv())
OpenAI(api_key = os.environ.get('OPENAI_API_KEY'))

Loading & Splitting Documents into Chunks

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_split_document():
    loader = PyPDFDirectoryLoader('Documents/')
    print
    docs = loader.load_and_split(RecursiveCharacterTextSplitter(separators=['\n\n','\n','.',','],
                                                chunk_size = 600,
                                                chunk_overlap = 100)
                                        )
    return docs

Converting Text Data to Vectors and Storing in Vector-DataBase 

In [ ]:
from langchain.vectorstores.chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
# from langchain.vectorstores.faiss import FAISS

docs = load_split_document() ## LOADING DATA AND SPLITTING INTO CHUNKS
vector_store = Chroma.from_documents(documents=docs ,embedding=OpenAIEmbeddings()) ### SAVING DATA INTO DB
# vector_store = FAISS.from_documents(documents=docs ,embedding=OpenAIEmbeddings()) ### SAVING DATA INTO DB

Adding Memory Element to the Chatbot

In [11]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

Defining Custom Prompt for our chatbot

In [12]:
from langchain.prompts import PromptTemplate
template = '''You are a HR Assistance Chatbot.You have to answer based on the given question and relevant context only.
                If you have been provided with the Chathistory make sure to only use to maintain flow of conversation.Do not mix context and Chathistory Together.
                Do not make things on your own. If the required information is not in the data provided just respond with I'm Sorry,I Don't Know. I am just an HR assisstant
Context : {context}
---
Chathistory : {chat_history}
---
Question :{question}
---
Answer :
'''
prompt_template = PromptTemplate(input_variables=['context','question','chat_history'],template=template)

In [13]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0,max_tokens=100)
chain = ConversationalRetrievalChain.from_llm(llm,memory = memory,retriever=vector_store.as_retriever(search_type="mmr",search_kwargs={'k': 2,'fetch_k': 4})
                                                ,combine_docs_chain_kwargs={"prompt": prompt_template})

Testing

In [14]:
chain.invoke({'question':'What do you know about fluid dynamics'})['answer']

"I'm Sorry, I Don't Know. I am just an HR assistant."

In [15]:
chain.run({'question':'What do you know about Devsloop'})

'Devsloop Technologies is a renowned software house committed to delivering cutting-edge software solutions. With a team of experienced professionals, Devsloop Technologies excels in various domains, including web development, mobile application development, AI and machine learning, and cloud computing. Their mission is to empower businesses by providing innovative and efficient technological solutions tailored to meet their unique needs. They also prioritize work-life balance, recognition and rewards for employees, and social activities to foster a sense of community.'

In [16]:
chain.run({'question':'How do i apply for an internship?'})

'Interested candidates can apply for an internship at Devsloop Technologies through our website by submitting their resume and a cover letter.'

In [17]:
chain.run({'question':'After this what is the procedure?'})

'After submitting your resume and cover letter on the website to apply for an internship at Devsloop Technologies, the next step would be the selection process. Shortlisted candidates will be invited for an interview and technical assessment to further evaluate their qualifications and fit for the internship position.'

In [18]:
chain.run({'question':'What skills do we learn during Internship'})

'During an internship at Devsloop Technologies, interns typically acquire practical skills in software development, project management, problem-solving, teamwork, communication, and time management. They also gain hands-on experience with the latest technologies and tools used in the software industry.'

In [19]:
chain.run({'question':'What about programming skills'})

'During an internship at Devsloop Technologies, interns typically learn programming skills related to software development, such as coding in languages like Java, Python, C++, or JavaScript. They also gain experience in working with databases, debugging, testing, and version control systems.'

Checking Memory of Chatbot

In [20]:
chain.memory.buffer

[HumanMessage(content='What do you know about fluid dynamics'),
 AIMessage(content="I'm Sorry, I Don't Know. I am just an HR assistant."),
 HumanMessage(content='What do you know about Devsloop'),
 AIMessage(content='Devsloop Technologies is a renowned software house committed to delivering cutting-edge software solutions. With a team of experienced professionals, Devsloop Technologies excels in various domains, including web development, mobile application development, AI and machine learning, and cloud computing. Their mission is to empower businesses by providing innovative and efficient technological solutions tailored to meet their unique needs. They also prioritize work-life balance, recognition and rewards for employees, and social activities to foster a sense of community.'),
 HumanMessage(content='How do i apply for an internship?'),
 AIMessage(content='Interested candidates can apply for an internship at Devsloop Technologies through our website by submitting their resume and